##**Notebook PC#11**

## Encoder-Decoder LSTM for Natural Language Processing.

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Ariel Góes de Castro <br>
**Aluno(a):** Francisco Germano Vogt

In [1]:
from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for i in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [3]:
seed(1)
n_samples =1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [4]:
from math import ceil
from math import log10

In [5]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [6]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [7]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [8]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [9]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [10]:
X,y = integer_encode(X,y,alphabet)

In [11]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [12]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)

    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [13]:
X,y = one_hot_encode(X,y,len(alphabet))

In [14]:
print(X,y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [15]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

In [16]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [17]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [18]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [19]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [20]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

2024-05-30 20:55:58.239600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-30 20:55:58.243423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-30 20:55:58.243534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-30 20:55:58.244112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                26400     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 75)            0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             25200     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            612       
 ibuted)                                                         
                                                                 
Total params: 52,212
Trainable params: 52,212
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

2024-05-30 20:56:02.426169: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


7500/7500 [==============================] - 39s 5ms/step - loss: 0.4040 - accuracy: 0.8781


In [22]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.023789, Accuracy: 99.500000


In [23]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

   3+6+3 = 12 (expect 12)
   7+5+2 = 14 (expect 14)
  10+6+4 = 20 (expect 20)
   5+9+2 = 16 (expect 16)
  2+4+10 = 16 (expect 16)
 7+10+10 = 27 (expect 27)
   2+3+8 = 13 (expect 13)
   4+1+8 = 13 (expect 13)
   1+3+5 =  9 (expect  9)
   9+6+1 = 16 (expect 16)


<font color="green">
Atividade (a) <br>
Como são gerados os dados de treinamento?
</font>

**Resposta:**

Vamos explicar as funções que são chamadas dentro da função `generate_data` para entender o processo:

```python
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y
```

#### Na função `random_sum_pairs` temos os seguintes parâmetros de entrada para gerar pares de somas aleatórios:

* _n_examples:_ o número de exemplos de pares entrada-saída (saída esperada/rotulada) a serem gerados;
* _n_numbers:_ o número de inteiros em cada entrada. Essa entrada é uma lista de tamanho fixo definida por `n_numbers`;
* _largest:_ O maior número que cada inteiro aleatório da entrada X pode assumir (definido por `largest`).

No exemplo abaixo, do próprio código:
```
seed(1)  # Set the seed for reproducibility
n_samples = 1  # Number of samples to generate
n_numbers = 2  # Number of numbers in each input list
largest = 10  # Largest possible number in the input list
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
```

Saída:
```
[[3, 10]] [13]
```

No caso acima, temos uma amostra gerada (`n_samples=1`), onde os elementos de X são a lista [3,10] de tamanho `n_numbers=2` e cada valor não ultrapassa o valor máximo de 10 (`largest=10`). Já a saída Y, representa a soma esperada/rotulada.


#### Na função `to_string`, converte-se a saída anterior:

Saída:
```
[' 3+10'] ['13']
```

Essa representação vai ser posteriormente utilizada na chamada seguinte (`integer_encode`).

#### Para a função `integer_encode` temos:

a declaração de uma alfabeto de símbolos que será utilizado pelo modelo: <br>
`alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']`

A função `integer_encode` recebe as string de entrada (X) e saída esperada (y) e converte cada símbolo de acordo com a posição de índices (começando em zero) da lista `alphabet`.

Por exemplo, a saída esperada no nosso exemplo seria:
```
[[11, 3, 10, 1, 0]] [[1, 3]]
```

onde 11 é o último índice da lista (' '), 3 é o próprio 3, 10 é o símbolo '+', e assim por diante.


## Para a função `one_hot_encode`:

Convertemos a saída numérica e para facilitar o aprendizado, converte-se cada símbolo que será utilizado em uma representação binária única, chamada one-hot-econding. Dessa forma, cada símbolo pode ser visto como uma classe separada e evitar falsas interpretações da rede neural que poderiam ocorrer caso utilizássemos as strings  (e.g., '0' visto como 0, '1' visto como 1, etc.)

Por fim, a última linha garante que tanto o X quanto y estejam em formato de array antes de retornar a saída da função `generate_data`.

#### Resumindo:
Geramos pares de números e a saída esperada para que o modelo aprenda a manipular os símbolos. Porém, realiza-se conversões intermediárias (e.g., `integer_encode`) para utilizarmos os dados como one-hot-encoding.


<font color="green">
Atividade (b) <br>
Como uma calculadora simples pode operar baseada no conceito de tradução de frases, ou seja, sem realizar operações algébricas?
</font>

**Resposta:** <br>

O modelo analisa as entradas como um conjunto de símbolos e saídas que devem ser corretamente previstas. Por exemplo, podemos tranformar a sequência "3+6" em outra sequência que representa a soma desses números, totalizando "9". Logo, o problema consiste em mapear uma sequência para outra sequência (a.k.a., seq2seq). 

A entrada é processada e convertida para um vetor de tamanho fixo, capturando as informações de toda a sequência de entrada. Logo, o vetor de contexto gera a sequência de saída (um símbolo por vez).

Durante o treinamento, o modelo aprende a reconhecer padrões em sequências, tokenizando as entradas (e.g., "3+6") como uma sequência de vectores one-hot-econded. O mesmo acontece com a saída "9".

Nesta arquitetura, temos os principais componentes:
* Camada LSTM: codifica a sequência de entrada em um vetor de comprimento fixo.
* Camada RepeatVector: repete o vetor codificado para corresponder ao comprimento da sequência de saída.
* Segunda camada LSTM: Decodifica o vetor repetido em uma sequência de saída.
* Camada TimeDistributed: aplica uma camada densa a cada intervalo de tempo da sequência de saída para produzir a saída final.

A cada iteração os pesos são ajustados. A camada `RepeatVector` cria uma sequência de comprimento fixo a partir do vetor de contexto codificado para o decodificador. Este vetor de contexto repetido é então alimentado na segunda camada LSTM. A função `softmax` transforma a saída da rede em probabilidades, que são esperadas pela função de perda categorical_crossentropy. Durante o treinamento, a perda de entropia cruzada medirá a diferença entre a distribuição de probabilidade prevista (da saída softmax) e a distribuição verdadeira (verdade fundamental codificada one-hot).
 
